In [50]:
import os
import zipfile
from yolo import YOLO

Using TensorFlow backend.


In [13]:
kaufland_files = zipfile.ZipFile('Processed_Data.zip','r')

In [14]:
kaufland_files.extractall()

In [15]:
kaufland_files = zipfile.ZipFile('Processed_XMLs.zip','r')

In [16]:
kaufland_files.extractall()

In [41]:
def loadFiles(fileName):
    result = []
    f = open(fileName, "r")

    for l in f:
        l = l.replace("C:\\Users\\c10670A\\Documents\\Python Scripts\\03. Projects\\Kaufland_Case\\", "/workspace/strabron/").replace("\\","/")
        #result.append(l.split(" ")[0])
        result.append(l)
    f.close()
    
    return result

In [44]:
files = ''.join(loadFiles("/workspace/strabron/Processed_XMLs/dev_data.txt")) + "\n" + ''.join(loadFiles("/workspace/strabron/Processed_XMLs/gen_data.txt"))

In [45]:
files

'/workspace/strabron/Processed_Data/train/IMG_20190305_160228_small.jpg 120,62,165,125,1 167,64,210,125,1 137,129,194,156,0 214,62,260,126,1 260,62,307,126,1 227,128,284,157,0 307,79,353,125,1 351,79,400,125,1 332,129,390,156,0 308,33,356,80,1 354,32,400,79,1 119,252,155,298,1 156,253,189,297,1 193,247,225,293,1 226,250,260,297,1 262,249,296,295,1 296,251,332,295,1 334,251,370,295,1 369,251,400,295,1\n/workspace/strabron/Processed_Data/train/IMG_20190305_160229_small.jpg 116,252,151,298,1 151,253,186,297,1 188,247,221,293,1 222,250,256,296,1 258,249,291,293,1 292,250,327,293,1 329,250,365,293,1 364,250,398,293,1 114,62,161,125,1 162,64,206,124,1 132,129,189,156,0 210,62,255,125,1 255,62,303,125,1 222,128,279,156,0 302,78,348,124,1 303,33,351,80,1 349,32,399,79,1 347,79,395,124,1 327,128,385,156,0\n/workspace/strabron/Processed_Data/train/IMG_20190305_160236_small.jpg 82,165,128,212,1 128,167,171,211,1 175,165,215,210,1 216,167,258,211,1 260,165,302,210,1 302,167,347,211,1 347,168,394,2

In [47]:
f = open("train.txt", "w")
f.write(files)
f.close()

In [48]:
files = ''.join(loadFiles("/workspace/strabron/Processed_XMLs/val_data.txt"))

In [49]:
f = open("val.txt", "w")
f.write(files)
f.close()

In [52]:
"""
Retrain the YOLO model for your own dataset.
"""

import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data



def get_classes(classes_path):
    '''loads the classes'''
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)


def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/tiny_yolo_weights.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)


In [53]:
annotation_path = 'train.txt'
validation_path = 'val.txt'
log_dir = 'logs/000/'
classes_path = 'model_data/voc_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (416,416) # multiple of 32, hw

is_tiny_version = len(anchors)==6 # default setting
if is_tiny_version:
    model = create_tiny_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/tiny_yolo_weights.h5')
else:
    model = create_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='logs/000/trained_weights_stage_1.h5') # make sure you know what you freeze

logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()


with open(validation_path) as f:
    lines_val = f.readlines()

np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines_val))
num_train = len(lines)

# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
if True:
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
    model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
    print('Unfreeze all of the layers.')

    batch_size = 32 # note that more GPU memory is required after unfreezing the body
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrapper(lines, batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines_val, batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=1,
        callbacks=[logging, checkpoint, reduce_lr, early_stopping])
    model.save_weights(log_dir + 'trained_weights_final.h5')

# Further training if needed.

Instructions for updating:
Colocations handled automatically by placer.
Create YOLOv3 model with 10 anchors and 2 classes.
Load weights logs/000/trained_weights_stage_1.h5.
Freeze the first 249 layers of total 252 layers.
Unfreeze all of the layers.
Train on 1141 samples, val on 75 samples, with batch size 32.
Epoch 1/1
35/35 [==============================] - 1718s 49s/step - loss: 62.6336 - val_loss: 55.0196


In [54]:
model.fit_generator(data_generator_wrapper(lines, batch_size, input_shape, anchors, num_classes),
    steps_per_epoch=max(1, num_train//batch_size),
    validation_data=data_generator_wrapper(lines_val, batch_size, input_shape, anchors, num_classes),
    validation_steps=max(1, num_val//batch_size),
    epochs=1,
    callbacks=[logging, checkpoint, reduce_lr, early_stopping])
model.save_weights(log_dir + 'trained_weights_final1.h5')

Epoch 1/1
35/35 [==============================] - 1495s 43s/step - loss: 47.6765 - val_loss: 46.6586


In [55]:
model.fit_generator(data_generator_wrapper(lines, batch_size, input_shape, anchors, num_classes),
    steps_per_epoch=max(1, num_train//batch_size),
    validation_data=data_generator_wrapper(lines_val, batch_size, input_shape, anchors, num_classes),
    validation_steps=max(1, num_val//batch_size),
    epochs=1,
    callbacks=[logging, checkpoint, reduce_lr, early_stopping])
model.save_weights(log_dir + 'trained_weights_final2.h5')

Epoch 1/1
35/35 [==============================] - 1504s 43s/step - loss: 43.4022 - val_loss: 44.2376


In [58]:
import sys
import argparse
from yolo import YOLO, detect_video
from PIL import Image
import cv2

ModuleNotFoundError: No module named 'cv2'

In [ ]:
def loadFiles(fileName):
    result = []
    f = open(fileName, "r")

    for l in f:
        l = l.replace("C:\\Users\\c10670A\\Documents\\Python Scripts\\03. Projects\\Kaufland_Case\\", "/workspace/strabron/").replace("\\","/")
        result.append(l.split(" ")[0])
    f.close()
    
    return result


def processImage(yolo, fileName, result):
    
    vid = cv2.VideoCapture(fileName)
    return_value, frame = vid.read()

    image = Image.fromarray(frame)
    
    image, out_boxes, out_scores, out_classes = yolo.detect_image(image)
    
    for x, c in enumerate(out_classes):
        top, left, bottom, right = out_boxes[x]
        top = str(max(0, int(top)))
        left = str(max(0, int(left)))
        bottom = str(min(320, int(bottom)))
        right = str(min(416, int(right)))
        box = [left, top, right, bottom]
        row = [fileName, str(out_classes[x]), str(out_scores[x])] + box
        result.append(",".join(row))

In [ ]:
files = loadFiles("/workspace/strabron/Processed_XMLs/dev_data.txt")
result = []
yolo = model#YOLO()
for fn in files:
    processImage(yolo, fn, result)
f = open("output.txt", "w")
f.write("\n".join(result))
f.close()